In [3]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [4]:
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext



In [5]:
!java -version

java version "1.8.0_321"
Java(TM) SE Runtime Environment (build 1.8.0_321-b07)
Java HotSpot(TM) 64-Bit Server VM (build 25.321-b07, mixed mode)


In [6]:
import pyspark

# number_cores = 1
# memory_gb = 8
# conf = (
#     pyspark.SparkConf()
# #         .master('spark://xxx.xxx.xx.xx:7077') \
#         .setMaster('local[{}]'.format(number_cores)) \
#         .set('spark.driver.memory', '{}g'.format(memory_gb))        
# )
# sc = pyspark.SparkContext(conf=conf)
# # sqlContext = SQLContext(sc)
# sqlContext = SQLContext(sc)
# #http://localhost:4040/jobs/

In [7]:
import raydp
import ray 

ray.init()
spark = raydp.init_spark(
  app_name = "raydp_spark",
  num_executors = 1,
  executor_cores = 1,
  executor_memory = "4GB"
)

2022-02-21 17:57:43,764	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265
(raylet) E0221 17:58:00.805181000 4561780224 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers
(raylet) E0221 17:58:00.837109000 4561780224 fork_posix.cc:76]                  Other threads are currently calling into gRPC, skipping fork() handlers


2022-02-21 17:58:17,914 WARN NativeCodeLoader [Thread-2]: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-02-21 17:58:18,117 INFO SecurityManager [Thread-2]: Changing view acls to: mk
2022-02-21 17:58:18,118 INFO SecurityManager [Thread-2]: Changing modify acls to: mk
2022-02-21 17:58:18,119 INFO SecurityManager [Thread-2]: Changing view acls groups to: 
2022-02-21 17:58:18,120 INFO SecurityManager [Thread-2]: Changing modify acls groups to: 
2022-02-21 17:58:18,120 INFO SecurityManager [Thread-2]: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(mk); groups with view permissions: Set(); users  with modify permissions: Set(mk); groups with modify permissions: Set()
2022-02-21 17:58:19,530 INFO Utils [Thread-2]: Successfully started service 'RAY_RPC_ENV' on port 60528.


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/Users/mk/opt/anaconda3/lib/python3.9/site-packages/ray/jars/ray_dist.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/Users/mk/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2022-02-21 17:58:32,173 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-7]: Registering app raydp_spark
2022-02-21 17:58:32,178 INFO RayAppMaster$RayAppMasterEndpoint [dispatcher-event-loop-7]: Registered app raydp_spark with ID app-20220221175832-0000


In [86]:
from ray.util.dask import ray_dask_get, enable_dask_on_ray, disable_dask_on_ray

enable_dask_on_ray()


In [ ]:
df = spark.createDataFrame(["Anna","Bob","Sue"], "string").toDF("firstname")

In [9]:
df.show()

+---------+
|firstname|
+---------+
|     Anna|
|      Bob|
|      Sue|
+---------+



In [10]:
names = ["Anna", "Bob", "Liam", "Olivia", "Noah", "Emma", "Oliver", "Ava", "Elijah", "Charlotte"]
class StudentRecord:
    def __init__(self, record_id, name):
        self.record_id = record_id
        self.name = name
    def __str__(self):
        return f'StudentRecord(record_id={self.record_id},data={self.name})'
    
num_records = len(names)
student_records = [StudentRecord(i, f'{names[i]}') for i in range(num_records)] 


In [11]:
student_records

In [12]:
df = spark.createDataFrame(student_records, ['name', 'id'])

In [13]:
df.show()

+---------+---+
|     name| id|
+---------+---+
|     Anna|  0|
|      Bob|  1|
|     Liam|  2|
|   Olivia|  3|
|     Noah|  4|
|     Emma|  5|
|   Oliver|  6|
|      Ava|  7|
|   Elijah|  8|
|Charlotte|  9|
+---------+---+



In [14]:
ray_dataset = ray.data.from_spark(df)

In [15]:
ray_dataset.show()

{'name': 'Anna', 'id': 0}
{'name': 'Bob', 'id': 1}
{'name': 'Liam', 'id': 2}
{'name': 'Olivia', 'id': 3}
{'name': 'Noah', 'id': 4}
{'name': 'Emma', 'id': 5}
{'name': 'Oliver', 'id': 6}
{'name': 'Ava', 'id': 7}
{'name': 'Elijah', 'id': 8}
{'name': 'Charlotte', 'id': 9}


In [16]:
from ray.util.dask import ray_dask_get, enable_dask_on_ray, disable_dask_on_ray
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd


In [22]:
import dask

In [ ]:
# ray.data.dataset.Dataset.to_dask

In [23]:
ddf_students = ray.data.dataset.Dataset.to_dask(ray_dataset) 

In [26]:
ddf_students.head()

,name,id
0,Anna,0
1,Bob,1
2,Liam,2
3,Olivia,3
4,Noah,4


In [60]:
from ray.util.dask import ray_dask_get


In [87]:
dask.config.get

<function dask.config.get(key, default='__no_default__', config={'temporary-directory': None, 'tokenize': {'ensure-deterministic': False}, 'dataframe': {'shuffle-compression': None, 'parquet': {'metadata-task-size-local': 512, 'metadata-task-size-remote': 16}}, 'array': {'svg': {'size': 120}, 'slicing': {'split-large-chunks': None}, 'chunk-size': '128MiB', 'rechunk-threshold': 4}, 'optimization': {'fuse': {'active': None, 'ave-width': 1, 'max-width': None, 'max-height': inf, 'max-depth-new-edges': None, 'subgraphs': None, 'rename-keys': True}}, 'scheduler': <function ray_dask_get at 0x7fa96ebf1e50>, 'distributed': {'version': 2, 'scheduler': {'allowed-failures': 3, 'bandwidth': 100000000, 'blocked-handlers': [], 'default-data-size': '1kiB', 'events-cleanup-delay': '1h', 'idle-timeout': None, 'transition-log-length': 100000, 'events-log-length': 100000, 'work-stealing': True, 'work-stealing-interval': '100ms', 'worker-ttl': None, 'pickle': True, 'preload': [], 'preload-argv': [], 'unkno

In [88]:
dsk_config_dump = dask.config.config.get('distributed')

In [91]:
dsk_config_dump.get('dashboard').get('link')

'{scheme}://{host}:{port}/status'

In [ ]:
# cluster.scheduler.services['dashboard'].server.address

In [ ]:
# larger dataset

In [34]:
url = "https://gender-pay-gap.service.gov.uk/viewing/download-data/2021"
from pyspark import SparkFiles


In [35]:
spark.sparkContext.addFile(url)

In [40]:
df = spark.read.csv("file://"+SparkFiles.get("2021"), header=True, inferSchema= True)

In [47]:
df.show(3)

+--------------------+----------+--------------------+--------+-------------+--------+---------------------+-----------------------+--------------------+----------------------+----------------+------------------+-----------------+-------------------+-----------------------+-------------------------+-----------------------+-------------------------+---------------+-----------------+--------------------+-------------------+------------+-----------+-------------------------+-------+-------------+
|        EmployerName|EmployerId|             Address|PostCode|CompanyNumber|SicCodes|DiffMeanHourlyPercent|DiffMedianHourlyPercent|DiffMeanBonusPercent|DiffMedianBonusPercent|MaleBonusPercent|FemaleBonusPercent|MaleLowerQuartile|FemaleLowerQuartile|MaleLowerMiddleQuartile|FemaleLowerMiddleQuartile|MaleUpperMiddleQuartile|FemaleUpperMiddleQuartile|MaleTopQuartile|FemaleTopQuartile|CompanyLinkToGPGInfo|  ResponsiblePerson|EmployerSize|CurrentName|SubmittedAfterTheDeadline|DueDate|DateSubmitted|
+-

In [42]:
ray_dataset = ray.data.from_spark(df)

In [46]:
ray_dataset.show(3)

{'EmployerName': '1509 GROUP', 'EmployerId': 15320.0, 'Address': 'Royal Grammar School, High Street, Guildford, Surrey, GU1 3BB', 'PostCode': 'GU1 3BB', 'CompanyNumber': '04104101', 'SicCodes': '85200,', 'DiffMeanHourlyPercent': None, 'DiffMedianHourlyPercent': None, 'DiffMeanBonusPercent': None, 'DiffMedianBonusPercent': None, 'MaleBonusPercent': None, 'FemaleBonusPercent': None, 'MaleLowerQuartile': None, 'FemaleLowerQuartile': None, 'MaleLowerMiddleQuartile': None, 'FemaleLowerMiddleQuartile': None, 'MaleUpperMiddleQuartile': None, 'FemaleUpperMiddleQuartile': None, 'MaleTopQuartile': None, 'FemaleTopQuartile': None, 'CompanyLinkToGPGInfo': None, 'ResponsiblePerson': None, 'EmployerSize': None, 'CurrentName': None, 'SubmittedAfterTheDeadline': None, 'DueDate': None, 'DateSubmitted': None}
{'EmployerName': '85310"', 'EmployerId': 18.0, 'Address': '16.0', 'PostCode': None, 'CompanyNumber': None, 'SicCodes': '0.0', 'DiffMeanHourlyPercent': 0.0, 'DiffMedianHourlyPercent': 36.0, 'DiffMea

In [49]:
from dask.distributed import Client
client = Client()

In [51]:
ddf_pay = ray.data.dataset.Dataset.to_dask(ray_dataset) 

In [52]:
ddf_pay.compute()

,EmployerName,EmployerId,Address,PostCode,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,...,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted
0,1509 GROUP,15320.0,"Royal Grammar School, High Street, Guildford, ...",GU1 3BB,04104101,"85200,",NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,"85310""",18.0,16.0,None,None,0.0,0.0,36.0,64.0,36.0,...,Less than 250,1509 GROUP,False,2022/04/05 00:00:00,2022/02/02 11:06:02,None,None,None,None,None
2,1LIFE MANAGEMENT SOLUTIONS LIMITED,687.0,"Ldh House St Ives Business Park, Parsons Green...",PE27 4AA,02566586,"93110,",NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,93130,NaN,None,None,None,None,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,"93290""",6.1,-35.3,55.5,-100.0,69.2,30.8,58.3,41.7,34.8,...,Less than 250,1LIFE MANAGEMENT SOLUTIONS LIMITED,False,2022/04/05 00:00:00,2022/02/01 12:19:55,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,ZEN INTERNET LIMITED,14058.0,"Sandbrook House Sandbrook Park, Sandbrook Way,...",OL11 1RY,03101568,61100,19.7,25.0,35.3,35.4,...,25.8,87.4,12.6,http://zen.co.uk,Sarah Howard-Illston (HR Manager),500 to 999,ZEN INTERNET LIMITED,False,2022/04/05 00:00:00,2021/08/11 19:12:27
3076,ZENITH MULTI ACADEMY TRUST,16487.0,"C/O The King John School Shipwrights Drive, Th...",SS7 1RQ,07559293,85310,7.8,11.9,NaN,NaN,...,75.0,49.5,50.5,None,None,Not Provided,ZENITH MULTI ACADEMY TRUST,False,2022/03/31 00:00:00,2022/02/18 12:02:12
3077,ZING LEISURE LIMITED,14068.0,"5 New Street Square, London, United Kingdom, E...",EC4A 3TW,05175354,56102,14.1,0.0,70.1,-231.6,...,46.3,63.0,37.0,None,Timothy Doubleday (CFO),250 to 499,ZING LEISURE LIMITED,False,2022/04/05 00:00:00,2021/08/18 09:37:29
3078,ZOPA BANK LIMITED,20751.0,"1st Floor Cottons Centre, 47-49 Tooley Street,...",SE1 2QG,10627575,64191,26.5,36.1,34.5,32.6,...,32.0,77.0,23.0,https://www.zopa.com/blog/article/gender-diver...,George Petrakis (Head of People Operations),250 to 499,ZOPA BANK LIMITED,False,2022/04/05 00:00:00,2022/01/27 16:51:42


In [45]:
ddf_pay.head(3)

,EmployerName,EmployerId,Address,PostCode,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,...,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted
0,1509 GROUP,15320.0,"Royal Grammar School, High Street, Guildford, ...",GU1 3BB,04104101,"85200,",NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,"85310""",18.0,16.0,None,None,0.0,0.0,36.0,64.0,36.0,...,Less than 250,1509 GROUP,False,2022/04/05 00:00:00,2022/02/02 11:06:02,None,None,None,None,None
2,1LIFE MANAGEMENT SOLUTIONS LIMITED,687.0,"Ldh House St Ives Business Park, Parsons Green...",PE27 4AA,02566586,"93110,",NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


,EmployerName,EmployerId,Address,PostCode,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,MaleLowerQuartile,FemaleLowerQuartile,MaleLowerMiddleQuartile,FemaleLowerMiddleQuartile,MaleUpperMiddleQuartile,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,float64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [54]:
def fillna(df):
    return df.fillna(value={"PostCode": "UNKNOWN"}).fillna(value=0)
    
new_df = ddf_pay.map_partitions(fillna)


In [55]:
new_df.compute()

,EmployerName,EmployerId,Address,PostCode,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,...,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted
0,1509 GROUP,15320.0,"Royal Grammar School, High Street, Guildford, ...",GU1 3BB,04104101,"85200,",0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,"85310""",18.0,16.0,UNKNOWN,0,0.0,0.0,36.0,64.0,36.0,...,Less than 250,1509 GROUP,False,2022/04/05 00:00:00,2022/02/02 11:06:02,0,0,0,0,0
2,1LIFE MANAGEMENT SOLUTIONS LIMITED,687.0,"Ldh House St Ives Business Park, Parsons Green...",PE27 4AA,02566586,"93110,",0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,93130,0.0,0,UNKNOWN,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,"93290""",6.1,-35.3,55.5,-100.0,69.2,30.8,58.3,41.7,34.8,...,Less than 250,1LIFE MANAGEMENT SOLUTIONS LIMITED,False,2022/04/05 00:00:00,2022/02/01 12:19:55,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,ZEN INTERNET LIMITED,14058.0,"Sandbrook House Sandbrook Park, Sandbrook Way,...",OL11 1RY,03101568,61100,19.7,25.0,35.3,35.4,...,25.8,87.4,12.6,http://zen.co.uk,Sarah Howard-Illston (HR Manager),500 to 999,ZEN INTERNET LIMITED,False,2022/04/05 00:00:00,2021/08/11 19:12:27
3076,ZENITH MULTI ACADEMY TRUST,16487.0,"C/O The King John School Shipwrights Drive, Th...",SS7 1RQ,07559293,85310,7.8,11.9,0.0,0.0,...,75.0,49.5,50.5,0,0,Not Provided,ZENITH MULTI ACADEMY TRUST,False,2022/03/31 00:00:00,2022/02/18 12:02:12
3077,ZING LEISURE LIMITED,14068.0,"5 New Street Square, London, United Kingdom, E...",EC4A 3TW,05175354,56102,14.1,0.0,70.1,-231.6,...,46.3,63.0,37.0,0,Timothy Doubleday (CFO),250 to 499,ZING LEISURE LIMITED,False,2022/04/05 00:00:00,2021/08/18 09:37:29
3078,ZOPA BANK LIMITED,20751.0,"1st Floor Cottons Centre, 47-49 Tooley Street,...",SE1 2QG,10627575,64191,26.5,36.1,34.5,32.6,...,32.0,77.0,23.0,https://www.zopa.com/blog/article/gender-diver...,George Petrakis (Head of People Operations),250 to 499,ZOPA BANK LIMITED,False,2022/04/05 00:00:00,2022/01/27 16:51:42


In [56]:
# Since there could be an NA in the index clear the partition / division information
new_df.clear_divisions()
new_df.compute()
narrow_df = new_df[["PostCode", "EmployerSize", "DiffMeanHourlyPercent"]]

In [57]:
grouped_df = narrow_df.groupby("PostCode")

In [59]:
avg_by_postalcode = grouped_df.mean()


avg_by_postalcode.compute()

,DiffMeanHourlyPercent
PostCode,
DE14 2EB,0.0
S65 1EG,13.2
-0.4,13.8
-0.6,51.4
-0.8,61.0
...,...
YO62 4HR,43.2
YO7 1PS,21.5
YO7 3JN,11.0


In [ ]:
sc.stop()

In [ ]:
sqlContext.stop()